# Generador de tomas Master

En este notebook se generan las tomas Master de las imágenes de calibración: Flat, Bias y Dark.

## Importación de módulos

[file_shortcuts](file_shortcuts.py) es el módulo en el cual se define la función que abre un conjunto de archivos .fits
y los convierte en un array de matrices. Cada matriz representa una imagen. También se guarda el tiempo de exposición.

[combination_functions](combination_functions.py) es el módulo en el cual se define las diferentes funciones que se
aplican para combinar imágenes.


In [8]:
import file_shortcuts as file
import combination_functions as func

## Apertura de archivos .fits

In [9]:
# Paths for each type of calibration image are defined
bias_path = "./data/Bias/Bias_Bias_"
darks_path = "./data/Darks/Darks/Dark_Dark_"
flats_path = "./data/Flats/Flat_Flat_Clear_"
flats_darks_path = "./data/Darks/Flat_Darks/Dark_Dark_"

# We define the number of images we have in each folder
n_b, n_d, n_f, n_f_d = 10, 10, 6, 5

# Generates an array with n images and a variable with the exposure time of the images
bits_bias, exp_time_bias = file.file_array_generator(bias_path, n_b)
bits_dark, exp_time_dark = file.file_array_generator(darks_path, n_d)
bits_flat, exp_time_flat = file.file_array_generator(flats_path, n_f)
bits_flat_dark, exp_time_flat_dark = file.file_array_generator(flats_darks_path, n_f_d)

# Método recomendado por la AAVSO

## Generación de Master Bias

Para la generación del archivo .fits Master del Bias se utiliza la media.

In [10]:
master_bias = func.average(bits_bias)
file.hdu_master_generator(master_bias,exp_time_bias,"Bias","AAVSO")

## Generación de Master Dark

Para la generación del archivo .fits Master del Dark se utiliza la mediana.

In [11]:
master_dark = func.median(bits_dark)
file.hdu_master_generator(master_dark, exp_time_dark,"Dark","AAVSO")

## Generación del Master Flat

Para la generación del archivo .fits Master del Flat, se utiliza la media.

Después se restan los Darks propios de los Flats, que previamente han de ser combinados para generar una
toma Master de los Darks de los Flats

In [12]:
master_flat = func.average(bits_flat) - func.median(bits_flat_dark)
file.hdu_master_generator(master_flat, exp_time_flat,"Flat","AAVSO")

# Método recomendado por Astropy: Desviación absoluta de la mediana

En este caso se estará utilizando la MAD para todos los tipos de archivo

In [13]:
master_bias_MAD = func.median_absolute_deviation(bits_bias)
master_dark_MAD = func.median_absolute_deviation(bits_dark)
master_flat_MAD = func.median_absolute_deviation(bits_flat) - func.median_absolute_deviation(bits_flat_dark)

file.hdu_master_generator(master_bias_MAD, exp_time_bias,"Bias_MAD","MAD")
file.hdu_master_generator(master_dark_MAD, exp_time_dark,"Dark_MAD","MAD")
file.hdu_master_generator(master_flat_MAD, exp_time_flat,"Flat_MAD","MAD")